In [46]:
#import libraries
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import csv
from urllib.request import Request
from urllib.error import URLError
from urllib.parse import urlparse
import os
import requests

In [47]:
# what collection to scrape, i used just this as reference so be careful of other pages seeming different

# collection_url = 'https://www.allrecipes.com/recipes/788/desserts/pies/apple-pie/'
# collection_url = 'https://www.allrecipes.com/recipes/23070/everyday-cooking/cookware-and-equipment/air-fryer/'
# collection_url = 'https://www.allrecipes.com/recipes/17123/main-dish/beef/beef-stroganoff/'
# collection_url = 'https://www.allrecipes.com/recipes/16932/salad/waldorf-salad/'
collection_url = 'https://www.allrecipes.com/recipes/199/holidays-and-events/valentines-day/'

## Getting Article Data

In [48]:
# getting article data
def get_recipe_name(soup):
    name = soup.find('h1', class_='article-heading') 
    return name.text.strip() if name else "No recipe name found"

def get_ingredients(soup):
    """
    Extract ingredients from the BeautifulSoup object.
    """
    ingredients_list = soup.find_all('li', class_='mm-recipes-structured-ingredients__list-item')
    ingredients_data = []
    
    for item in ingredients_list:
        quantity = item.find('span', {'data-ingredient-quantity': 'true'})
        unit = item.find('span', {'data-ingredient-unit': 'true'})
        name = item.find('span', {'data-ingredient-name': 'true'})
        
        ingredients_data.append({
            'quantity': quantity.text.strip() if quantity else None,
            'unit': unit.text.strip() if unit else None,
            'name': name.text.strip() if name else None,
        })
    
    return ingredients_data

def get_steps(soup):
    """
    Extract steps from the BeautifulSoup object as an ordered array.
    """
    # Find the ordered list (ol) containing the steps
    steps_list = soup.find('ol', {'id': 'mntl-sc-block_1-0'})
    
    # Check if the steps list is found
    if not steps_list:
        return []
    
    # Find all <li> elements within the <ol>
    step_items = steps_list.find_all('li')
    
    # Extract text from each <p> tag in the <li> elements
    steps = []
    for item in step_items:
        step_text = item.find('p')  # Find the <p> tag inside the <li>
        if step_text:
            steps.append(step_text.text.strip())  # Add the cleaned text to the list
    
    return steps

In [49]:
def get_nutrition_data(soup):
    """
    Extracts both summary nutrition values (calories, fat, carbs, protein) and servings/calories
    from the nutrition section of the BeautifulSoup object.
    """
    nutrition_data = {}
    
    # Extract summary nutrition (calories, fat, carbs, protein) from the nutrition facts summary section
    nutrition_summary = {}
    nutrition_table = soup.find('table', class_='mm-recipes-nutrition-facts-summary__table')
    
    if nutrition_table:
        rows = nutrition_table.find_all('tr', class_='mm-recipes-nutrition-facts-summary__table-row')
        
        for row in rows:
            value = row.find_all('td')[0].text.strip()  # Nutrient value (first <td>)
            name = row.find_all('td')[1].text.strip()   # Nutrient name (second <td>)
            
            if name.lower() in ['calories', 'fat', 'carbs', 'protein']:
                nutrition_summary[name.lower()] = value
    
    # Add the summary nutrition values to the nutrition_data dictionary
    nutrition_data.update(nutrition_summary)
    
    # Extract servings and calories from the main nutrition section
    servings_row = soup.find('tr', class_='mm-recipes-nutrition-facts-label__servings')
    if servings_row:
        servings_label = servings_row.find('span', class_='mm-recipes-nutrition-facts-label__table-head-pretext')
        if servings_label:
            servings_value = servings_label.find_next('span')  # Get the next sibling <span>
            if servings_value:
                nutrition_data['servings'] = servings_value.text.strip()
    
    calories_row = soup.find('tr', class_='mm-recipes-nutrition-facts-label__calories')
    if calories_row:
        calories_label = calories_row.find('span', class_='mm-recipes-nutrition-facts-label__table-head-pretext')
        if calories_label:
            calories_value = calories_label.find_next('span')  # Get the next sibling <span>
            if calories_value:
                nutrition_data['calories'] = calories_value.text.strip()
    
    return nutrition_data


In [ ]:
def fetch_recipe_data(url):
    """
    Parent function to fetch and organize recipe data.
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    req = Request(url, headers=headers)
    
    # Fetch the page
    response = urlopen(req)
    html = response.read().decode('utf-8')
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Fetch data from child functions
    recipe_data = {
        'name': get_recipe_name(soup),
        'ingredients': get_ingredients(soup),
        'steps': get_steps(soup),
        'nutrition': get_nutrition_data(soup),
        'link': url
    }
    
    return recipe_data

#### Example Usage

In [51]:
# To test feel free to take an example page like this and just paste it here

# url = 'https://www.allrecipes.com/air-fryer-baked-yams-recipe-8737640'
url = 'https://www.allrecipes.com/recipe/220619/real-homemade-bagels/'
# url = 'https://www.allrecipes.com/recipe/15683/dutch-apple-pie-with-oatmeal-streusel/'

# test with these urls

recipe_data = fetch_recipe_data(url)

print(recipe_data)

{'name': 'Real Homemade Bagels', 'ingredients': [{'quantity': '1 ¼', 'unit': 'cups', 'name': 'water'}, {'quantity': '4 ½', 'unit': 'cups', 'name': 'bread flour'}, {'quantity': '3', 'unit': 'tablespoons', 'name': 'white sugar'}, {'quantity': '1', 'unit': 'teaspoon', 'name': 'salt'}, {'quantity': '2', 'unit': 'tablespoons', 'name': 'vegetable oil'}, {'quantity': '1', 'unit': 'tablespoon', 'name': 'instant yeast'}, {'quantity': '4', 'unit': 'quarts', 'name': 'water'}, {'quantity': '1', 'unit': 'cup', 'name': 'honey (Optional)'}, {'quantity': '2', 'unit': 'tablespoons', 'name': 'poppy seeds (Optional)'}, {'quantity': '2', 'unit': 'tablespoons', 'name': 'sesame seeds (Optional)'}, {'quantity': '2', 'unit': 'tablespoons', 'name': 'dried onion flakes (Optional)'}, {'quantity': '1', 'unit': 'tablespoon', 'name': 'coarse salt (Optional)'}], 'steps': ['Make bagels: Combine water, flour, sugar, salt, vegetable oil, and yeast in the bowl of a stand mixer fitted with the dough hook. Mix on low spee

In [52]:
def print_recipe_formatted(recipe_data):
    print("\nRecipe Name:", recipe_data['name'])
    print("\nIngredients:")
    for ingredient in recipe_data['ingredients']:
        print(ingredient)
    print("\nSteps:")
    for i, step in enumerate(recipe_data['steps'], 1):
        print(f"{i}. {step}")
    print("\nNutrition Data:")
    for key, value in recipe_data['nutrition'].items():
        print(f"{key.capitalize()}: {value}")
    print("-------------------------------------------")

In [53]:
recipe_data = fetch_recipe_data(url)

print_recipe_formatted(recipe_data)


Recipe Name: Real Homemade Bagels

Ingredients:
{'quantity': '1 ¼', 'unit': 'cups', 'name': 'water'}
{'quantity': '4 ½', 'unit': 'cups', 'name': 'bread flour'}
{'quantity': '3', 'unit': 'tablespoons', 'name': 'white sugar'}
{'quantity': '1', 'unit': 'teaspoon', 'name': 'salt'}
{'quantity': '2', 'unit': 'tablespoons', 'name': 'vegetable oil'}
{'quantity': '1', 'unit': 'tablespoon', 'name': 'instant yeast'}
{'quantity': '4', 'unit': 'quarts', 'name': 'water'}
{'quantity': '1', 'unit': 'cup', 'name': 'honey (Optional)'}
{'quantity': '2', 'unit': 'tablespoons', 'name': 'poppy seeds (Optional)'}
{'quantity': '2', 'unit': 'tablespoons', 'name': 'sesame seeds (Optional)'}
{'quantity': '2', 'unit': 'tablespoons', 'name': 'dried onion flakes (Optional)'}
{'quantity': '1', 'unit': 'tablespoon', 'name': 'coarse salt (Optional)'}

Steps:
1. Make bagels: Combine water, flour, sugar, salt, vegetable oil, and yeast in the bowl of a stand mixer fitted with the dough hook. Mix on low speed until dough

## Crawling

#### Scraping 1 collection

In [54]:
def get_recipe_urls(page_url):
    """
    Fetches all recipe URLs from the main page (the one containing a list of recipe links).
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    req = Request(page_url, headers=headers)
    
    # Fetch the page
    response = urlopen(req)
    html = response.read().decode('utf-8')
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find all recipe links in the page (based on the mntl card item structure)
    recipe_links = []
    recipe_cards = soup.find_all('a', class_='mntl-card-list-items')  # This will depend on the actual class name

    for card in recipe_cards:
        link = card.get('href')
        if link and link not in recipe_links:
            recipe_links.append(link)
    
    return recipe_links

In [55]:
def scrape_recipes_from_page(page_url):
    """
    Scrapes all recipes from a given page, fetches the data, and returns an array of recipe data.
    """
    recipe_data_list = []
    
    # Step 1: Get all recipe URLs from the main page
    recipe_urls = get_recipe_urls(page_url)
    
    # Print the count of recipe links found
    print(f"Found {len(recipe_urls)} recipe links")
    
    counter = 1
    
    # Step 2: Iterate over the URLs and fetch the recipe data
    for recipe_url in recipe_urls:
        print(f"({counter}/ {len(recipe_urls)}) Fetching data for: {recipe_url}")
        
        # Fetch and organize the recipe data
        recipe_data = fetch_recipe_data(recipe_url)
        
        # Append the recipe data to the list
        recipe_data_list.append(recipe_data)
        
        counter += 1
    
    return recipe_data_list

#### Usage and turning into CSV

In [56]:
page_url = collection_url
recipes = get_recipe_urls(page_url)

In [57]:
recipes = scrape_recipes_from_page(page_url)

Found 71 recipe links
(1/ 71) Fetching data for: https://www.allrecipes.com/gallery/cute-and-easy-valentines-day-treats/
(2/ 71) Fetching data for: https://www.allrecipes.com/article/how-to-make-chocolate-covered-strawberries/
(3/ 71) Fetching data for: https://www.allrecipes.com/gallery/great-ways-to-make-shrimp-scampi/
(4/ 71) Fetching data for: https://www.allrecipes.com/gallery/fancy-valentines-day-dessert-recipes/
(5/ 71) Fetching data for: https://www.allrecipes.com/pomegranate-fizz-recipe-8724216
(6/ 71) Fetching data for: https://www.allrecipes.com/copycat-philadelphia-strawberry-cheesecake-bites-8580174
(7/ 71) Fetching data for: https://www.allrecipes.com/chocolate-mousse-for-beginners-recipe-8574786
(8/ 71) Fetching data for: https://www.allrecipes.com/heart-shaped-pizza-recipe-8430817
(9/ 71) Fetching data for: https://www.allrecipes.com/mini-heart-shaped-naan-pizzas-recipe-8422932
(10/ 71) Fetching data for: https://www.allrecipes.com/recipe/150033/gorgonzola-and-wild-mush

In [58]:
# Print the results
for recipe in recipes:
    print_recipe_formatted(recipe)


Recipe Name: 11 Cute and Easy Valentine's Day Treats

Ingredients:

Steps:

Nutrition Data:
-------------------------------------------

Recipe Name: How to Make the Best Chocolate-Covered Strawberries

Ingredients:

Steps:

Nutrition Data:
-------------------------------------------

Recipe Name: 14 Great Ways to Make Shrimp Scampi

Ingredients:

Steps:

Nutrition Data:
-------------------------------------------

Recipe Name: 18 Fancy Valentine's Day Dessert Recipes

Ingredients:

Steps:

Nutrition Data:
-------------------------------------------

Recipe Name: Pomegranate Fizz

Ingredients:
{'quantity': '5', 'unit': None, 'name': 'basil leaves'}
{'quantity': '1', 'unit': 'tablespoon', 'name': 'agave syrup or honey'}
{'quantity': '1', 'unit': 'fluid ounce', 'name': 'lime juice'}
{'quantity': '3', 'unit': 'fluid ounces', 'name': 'pomegranate juice'}
{'quantity': '4', 'unit': 'fluid ounces', 'name': 'club soda'}
{'quantity': None, 'unit': None, 'name': 'ice cubes,'}
{'quantity': '1', 

## Saving to CSV

In [59]:
def save_recipes_to_csv(page_url):
        # Parse the URL
    parsed_url = urlparse(page_url)
    path_parts = parsed_url.path.strip('/').split('/')
    last_part = path_parts[-1]  # Extract the last part of the URL

    # Dynamically generate the CSV filename
    csv_filename = f"{last_part}-recipes.csv"
    
    # Initialize CSV writer
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['name', 'link', 'ingredients', 'steps', 'nutrition']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        # recipe_links = get_recipe_urls(page_url)  # Assuming this function returns the list of recipe links

        # Loop through the recipe links
        for recipe_data in recipes:           
            # Ensure ingredients are all strings
            ingredients = ', '.join([str(ingredient) for ingredient in recipe_data['ingredients']])

            # Ensure steps are all strings
            steps = '. '.join([str(step) for step in recipe_data['steps']])

            # Ensure nutrition is a string, if it's a dictionary
            nutrition = ', '.join([f"{key}: {value}" for key, value in recipe_data['nutrition'].items()])

            # Write the recipe data to CSV
            writer.writerow({
                'name': recipe_data['name'],
                'link': recipe_data['link'],
                'ingredients': ingredients,
                'steps': steps,
                'nutrition': nutrition
            })


In [60]:
save_recipes_to_csv(page_url)